In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sklearn
import warnings
warnings.filterwarnings('ignore')

In [11]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/PROYECTO FINAL UTN/data/raw/telco.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

X = df.drop('Churn', axis=1)
y = df['Churn']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

if 'SeniorCitizen' in numerical_features:
    numerical_features.remove('SeniorCitizen')
    categorical_features.append('SeniorCitizen')

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)



numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])


X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)



print("Shape de X_train:", X_train_processed.shape)
print("Shape de X_test:", X_test_processed.shape)
print("Shape de y_train_encoded:", y_train_encoded.shape)
print("Shape de y_test_encoded:", y_test_encoded.shape)

Shape de X_train: (5634, 5680)
Shape de X_test: (1409, 5680)
Shape de y_train_encoded: (5634,)
Shape de y_test_encoded: (1409,)


In [13]:
def get_feature_names(column_transformer):
    """
    Get feature names from a ColumnTransformer.
    Based on: https://stackoverflow.com/questions/53567595/get-column-names-after-applying-sklearn-columntransformer

    Args:
        column_transformer: The fitted ColumnTransformer.

    Returns:
        A list of feature names.
    """
    output_features = []

    for name, transformer, original_features in column_transformer.transformers_:
        if transformer == 'drop' or transformer == 'passthrough':
            continue
        if hasattr(transformer, 'get_feature_names_out'):
            if isinstance(transformer, Pipeline):
                names = transformer.steps[-1][1].get_feature_names_out(original_features)
            else:
                names = transformer.get_feature_names_out(original_features)
            output_features.extend(names)
        elif isinstance(transformer, SimpleImputer):
            output_features.extend(original_features)
        elif isinstance(transformer, StandardScaler):
            output_features.extend(original_features)
        else:
            print(f"Warning: Unknown transformer type {type(transformer)}. Feature names may not be accurate.")
            output_features.extend(original_features)

    return output_features

feature_names_processed = get_feature_names(preprocessor)

print("Nombres de las variables después del preprocesamiento:")
print(feature_names_processed[:10])
print(f"Total de variables: {len(feature_names_processed)}")

Nombres de las variables después del preprocesamiento:
['tenure', 'MonthlyCharges', 'TotalCharges', 'customerID_0002-ORFBO', 'customerID_0003-MKNFE', 'customerID_0004-TLHLJ', 'customerID_0011-IGKFF', 'customerID_0013-EXCHZ', 'customerID_0013-MHZWF', 'customerID_0013-SMEOE']
Total de variables: 5680


In [14]:
if hasattr(X_train_processed, 'toarray'):
    X_train_processed = X_train_processed.toarray()
if hasattr(X_test_processed, 'toarray'):
    X_test_processed = X_test_processed.toarray()

feature_names_processed = get_feature_names(preprocessor)

X_train_df = pd.DataFrame(X_train_processed, columns=feature_names_processed)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names_processed)
y_train_df = pd.DataFrame(y_train_encoded, columns=['Churn_encoded'])
y_test_df = pd.DataFrame(y_test_encoded, columns=['Churn_encoded'])

X_train_df.to_csv('/content/drive/MyDrive/PROYECTO FINAL UTN/data/processed/X_train_processed.csv', index=False)
X_test_df.to_csv('/content/drive/MyDrive/PROYECTO FINAL UTN/data/processed/X_test_processed.csv', index=False)
y_train_df.to_csv('/content/drive/MyDrive/PROYECTO FINAL UTN/data/processed/y_train_encoded.csv', index=False)
y_test_df.to_csv('/content/drive/MyDrive/PROYECTO FINAL UTN/data/processed/y_test_encoded.csv', index=False)

print("Datos preprocesados guardados en la carpeta 'processed' de Google Drive.")

Datos preprocesados guardados en la carpeta 'processed' de Google Drive.


In [15]:
import pickle

with open('/content/drive/MyDrive/PROYECTO FINAL UTN/data/processed/preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

print("Preprocessor guardado en la carpeta 'processed' de Google Drive.")

Preprocessor guardado en la carpeta 'processed' de Google Drive.
